# Práctica 2. Gestión y explotación de datos

### Realizada por Fernando Lucas Ruiz

## 1. Importar librerias

In [1]:
from pathlib import Path
import re
import csv
import requests, sys
import mysql.connector


## 2. Funciones

Funcion para cargar los datos de los ficheros csv

In [2]:
def load_runs(ruta):
     # Guardar la ruta donde se encuentran los ficheros csv con los run
     base_path = Path(ruta) 

     diccionario_resultado = {} # diccionario vacio donde meteremos los datos de los csv

     
     for file in base_path.rglob("*.csv"):

          # guardar el nombre del run 
          er=re.compile("(run)+[0-9]")  
          run=er.search(str(file)).group()

          # Búsqueda de delimitador para abrir correctamente los archivos csv
          # para ello solamente abrimos la primera fila y buscamos el delimitador que lo utilizaremos en la siguiente estructura de control
          with open(file, mode = "r") as csv_file:
               csv_reader = csv.reader(csv_file)
               contador = 0
               for row in csv_reader:
                    if contador == 0:
                         if ";" in row[0]:
                              delimitador = ";"
                              contador +=1
                              continue
                         elif "," in row[0]:
                              delimitador = ";"
                              contador +=1
                              continue
                         elif "|" in row[0]:
                              delimitador = ";"
                              contador +=1
                              continue
                         elif "#" in row[0]:
                              delimitador = ";"
                              contador +=1
                              continue
                         else:
                              delimitador = '\t'
                              contador +=1
                    else:
                         break

          # Guardar los datos en diccionario con el delimitador correcto
          with open(file, mode = "r") as csv_file:
                    csv_reader = csv.DictReader(csv_file, delimiter = delimitador)
                    lista_resultado = []
                    for row in csv_reader:
                         lista_resultado.append(row)

          diccionario_resultado[run] = lista_resultado
     return diccionario_resultado

Función para buscar los ensemble IDs en función del gene symbol que tenga un gen específico.

In [3]:
def buscar_id_ensembl(gene_symbol):

     # conexión con el servidor dependiendo del gene_symbol específico
     server = "https://rest.ensembl.org"
     ext = "/xrefs/symbol/homo_sapiens/"+gene_symbol+ "?"
     
     r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
     
     if not r.ok:
          return None
     
     # Búsqueda dentro del decoded el ensembl ID
     decoded = r.json()
     for id in decoded:
          if id["id"].startswith("ENSG"):
               id_ensembl = id["id"]
               return id_ensembl

Función para buscar el gene symbol en función del ensembl ID que tenga un gen específico.

In [4]:
def buscar_gene_symbol(ensembl_id):
     
     # Conectar con el servidor para descargar la información en decoded
     server = "https://rest.ensembl.org"
     ext = "/xrefs/id/"+ensembl_id+"?"
     
     r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
     
     if not r.ok:
          r.raise_for_status()
          sys.exit()
     
     decoded = r.json()

     # La información del gene_symbol se encuentra en la base de datos EntreGene con el nombre display_id
     for id in decoded:
          if id["dbname"] == "EntrezGene":
               for i in id:
                    if i.startswith("display_id"):
                         gene_symbol = id[i]

     return gene_symbol

Función para conectarme con la base de datos de MySQL

In [5]:
def conectar_mysql(host, user, password, database) :
    #Usamos el mecanismo de excepciones para controlar que todo es correcto
    try :
        mydb = mysql.connector.connect(
            host=host,
            user=user, #Usuario
            password=password,#contraseña
            database=database #esquema
        )
        return mydb
    except :
        return None

## 3. Conexión y envio de datos a la base de datos

Lo primero de todo descargamos los datos en la variable "estructura_datos_runs" con la función load_runs

In [6]:
estructura_datos_runs = load_runs('/Users/fernandolucasruiz/Library/CloudStorage/OneDrive-UNIVERSIDADDEMURCIA/Documentos/Fernando/Master Bioinformatica/Asignaturas/Sistemas Bioinformáticos/Bloque2_gestion de datos/tarea2_2/')

Ahora con la función buscar_id_ensembl, metemos en los datos cargados el nombre del gen en formato ensembl

In [7]:
for run in estructura_datos_runs:
     for i in estructura_datos_runs[run]:
          ensembl_id=buscar_id_ensembl(i["gene"])
          i["ensembl"] = ensembl_id

Con este bloque de código pretendo actualizar la tabla b3b2_gen de la base de datos ya que en ella le falta la información de gene_symbol en su columna.

In [ ]:
# Hacemos conexión con la base de datos
conexion = conectar_mysql("dayhoff.inf.um.es", "alubio11", "alubio11PW-", "alubio11")

# Extraigo la columna gene_ensembl que es donde están todos los nombres de los genes en formato ensembl
mycursor = conexion.cursor()
mycursor.execute("SELECT gene_ensembl FROM b3b2_gen")
myresult = mycursor.fetchall()

# Plantilla de inserción
update_sql = "UPDATE b3b2_gen SET gene_symbol = %s WHERE gene_ensembl = %s"

# Con esta estructura de control voy pasando gen por gen y buscando su gene_symbol
for ens in myresult:
    symbol= buscar_gene_symbol(ens[0]) # ens[0] porque los nombress se descargan como tuplas
    mycursor.execute(update_sql, (symbol, ens[0])) # ejecutamos la actualización introduciendo el gene_symbol donde está su gene_ensembl específico

conexion.commit() #enviamos a la base de datos nuestra petición
mycursor.close()
conexion.close() #cerramos la conexión

Bloque de código en el cual actualizamos las listas b3b2_run, b3b2_expresion, b3b2_muestra_run

In [ ]:
# conexión con la base de datos
conexion = conectar_mysql("dayhoff.inf.um.es", "alubio11", "alubio11PW-", "alubio11") 
mycursor = conexion.cursor()

# Plantillas de inserción
run_sql = "INSERT INTO b3b2_run (nombre) VALUES (%s)"
expresion_insert = "INSERT INTO b3b2_expresion (muestra, gen, run, expresion) VALUES (%s, %s, %s, %s)"
muestra_run_insert = "INSERT INTO b3b2_muestra_run (muestra, run) VALUES (%s, %s)"

# Genero dos listas vacias para llevar el control de las muestras y los runs que pasamos para evitar algún tipo de fallo o duplicación
lista_muestras = []
lista_run = []

# Estructura de control para leer los datos almacenados en la variable estructura_datos_runs con los runs, muestras y expresiones
for run in estructura_datos_runs:

    # Con este bloque pretendo que solamente la primera pasada del run lo lance la la lista b3b2_run de la base de datos
    if run not in lista_run:
        mycursor.execute(run_sql, (run,))
        conexion.commit()
        lista_run.append(run)

    # Empezamos a leer los genes, muestras y expresiones
    for i in estructura_datos_runs[run]:
        ensembl_gen = i["ensembl"] # guardo el ensemble ID

        # Hago esta linea porque había un gen que me daba problemas porque era un novel_gen y decidí no incluirlo en la base de datos
        # intenté descargarme la información con la función buscar_gen_symbol pero no tenía ninguna información.
        mycursor.execute("SELECT COUNT(*) FROM b3b2_gen WHERE gene_ensembl = %s", (ensembl_gen,))
        if mycursor.fetchone()[0] == 0:
            continue
        
        # Bloque en el que ejecuto el envio de la información a la lista b3b2_expresión
        for sample, exp in i.items():
            if sample != "gene" and sample != "ensembl":
                muestra = sample
                expresion = exp
                mycursor.execute(expresion_insert, (muestra, ensembl_gen, run, expresion))

                # Bloque en el que ejecuto el envio de la información a la lista b3b2_muestra_run
                if sample not in lista_muestras:
                    mycursor.execute(muestra_run_insert, (muestra, run))
                    lista_muestras.append(sample)


conexion.commit() # enviamos la información a la base de datos
mycursor.close()
conexion.close() # cerramos la conexión con la base de datos
